In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import flopy
import pyemu
import shutil
import geopandas as gpd
from pathlib import Path
from mf6rtm import utils, mup3d
from flopy.utils import  cvfdutil
from vorflow import ConceptualMesh, MeshGenerator, VoronoiTessellator
from shapely import box, Point, LineString
# from shapely.geometry import LineString
from herebedragons import *

# Background

# Grid and domain

In [ ]:
domain = gpd.read_file(Path('data', 'domain.gpkg'))
Ly = domain.geometry.total_bounds[3] - domain.geometry.total_bounds[1]
Lx = domain.geometry.total_bounds[2] - domain.geometry.total_bounds[0]

xul = domain.geometry.total_bounds[0]
yul = domain.geometry.total_bounds[1]

domain_ext = box(xul, yul, xul + Lx, yul +  Ly)
domain_ext

In [ ]:
wells = pd.read_csv(Path('data', 'wells.csv'))
refinement = gpd.read_file(Path('data', 'refinement.gpkg'))

obsloc = pd.read_csv(Path('data', 'obs_loc.csv'))
obsloc.head()

wells

In [ ]:

geom = refinement.geometry[0]
minx, miny, maxx, maxy = geom.bounds

# Create line from bottom-left to bottom-right of top edge
top_line = LineString([(minx, maxy), (maxx, maxy)])
bottom_line = LineString([(minx, miny), (maxx, miny)])

In [ ]:
background_lc = 25.0

blueprint = ConceptualMesh()
blueprint.add_polygon(domain_ext, zone_id=1)#,border_density=100,dist_max_in=background_lc)

blueprint.add_line(bottom_line, line_id="Refinement-Line",
                     resolution=5,
                        dist_max=1*background_lc,
                        # z_order=3
                        )

for wid in wells.index:
    blueprint.add_point(Point(wells.loc[wid, 'x'], wells.loc[wid, 'y']), point_id=f"{wells.loc[wid, 'name']}", 
                    resolution=2,
                    dist_max=1.5*background_lc)
    print(f"Added well {wells.loc[wid, 'name']} at ({wells.loc[wid, 'x']}, {wells.loc[wid, 'y']})")
clean_polys, clean_lines, clean_pts = blueprint.generate()

# 2. Mesh Generation
mesher = MeshGenerator(background_lc=background_lc, verbosity=0)
mesher.generate(clean_polys, clean_lines, clean_pts)

# 3. Voronoi Conversion
tessellator = VoronoiTessellator(mesher, blueprint, clip_to_boundary=True)
grid_gdf = tessellator.generate()

In [ ]:
grid_gdf

In [ ]:
fig, ax = plt.subplots(1, 1)

grid_gdf.plot(
    ax=ax,
    alpha=0.5,
    edgecolor='k',
    linewidth=0.2
)
ax.scatter(
    wells['x'],
    wells['y'],
    color='red',
    marker='.',
    s=100,
    label='Wells'
)
ax.legend()

ax.set_aspect('equal')
fig.tight_layout()

In [ ]:
from vorflow.utils import *

gdf = calculate_mesh_quality(grid_gdf,calc_ortho=True)
# gdf.plot(column='drift_ratio', cmap='viridis', legend=True, figsize=(10,8),vmax=.25, aspect='equal')
summarize_quality(gdf)

grid_gdf.to_file(Path("data", "mf6_grid.shp"))

In [ ]:
verts, iverts = cvfdutil.shapefile_to_cvfd('data/mf6_grid.shp')
verts
# xcyc
gridprops = cvfdutil.get_disv_gridprops(verts, iverts, xcyc=None)
gridprops

In [ ]:
def make_gwf(tracer = 'Cl', mup3d_m = None):

    flow_modelname = "gwf"
    nper = 39  # Number of stress periods
    perioddata= [(2, 2, 1), (4, 4, 1), (4, 4, 1), (4, 4, 1), (7, 7, 1),
                (7, 7, 1), (7, 7, 1), (7, 7, 1), (14, 14, 1), (14, 14, 1), 
                (15, 15, 1), (13, 13, 1), (14, 14, 1), (14, 14, 1), (14, 14, 1), 
                (21, 21, 1), (35, 35, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
                (28, 28, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
                (35, 35, 1), (35, 35, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
                (35, 35, 1), (35, 35, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
                (28, 28, 1), (35, 35, 1), (35, 35, 1),(28, 28, 1)]


    sim = flopy.mf6.MFSimulation(sim_name=f"{flow_modelname}", version='mf6', sim_ws=ws)
    utils.prep_bins(ws, get_only=['mf6', 'libmf6', 'pestpp-ies', 'pestpp-mou'])
    # ?utils.prep_bins

    # specify tdis
    tdis = flopy.mf6.ModflowTdis(sim, pname="tdis", time_units="DAYS", 
                                    nper=nper, perioddata=perioddata)
    outer_dvclose = 1e-5
    inner_dvclose = 1e-5
    ims = flopy.mf6.ModflowIms(sim, 
                            #    pname="ims", 
                            complexity="complex",
                            outer_dvclose=outer_dvclose,
                            inner_dvclose=inner_dvclose,
                            filename=f"{flow_modelname}.ims")
    sim.register_ims_package(ims, 
                                [flow_modelname])

    # start model build to refine 
    model_nam_file = "{}.nam".format(flow_modelname)
    gwf = flopy.mf6.ModflowGwf(sim, modelname=flow_modelname, 
                                model_nam_file=model_nam_file, exe_name='mf6')

    disv = flopy.mf6.ModflowGwfdisv(gwf,
                                    nlay=12,
                                    ncpl=gridprops['ncpl'],
                                    nvert=gridprops['nvert'],
                                    vertices=gridprops['vertices'],
                                    cell2d=gridprops['cell2d'],
                                    top=-273.0,
                                    botm=0.0,
                                    filename=f"{flow_modelname}.disv")

    botms = get_botms(gwf, ws)
    disv.botm.set_data(botms)
    disv.set_all_data_external()

    nlay = disv.nlay.get_data()
    ncpl = disv.ncpl.get_data()

    ihead = 0 #(meters)
    strt = ihead * np.ones((nlay, ncpl))
    ic = flopy.mf6.ModflowGwfic(gwf, pname="ic", strt=strt)
    ic.set_all_data_external()

    npf = flopy.mf6.ModflowGwfnpf(
            gwf,
            icelltype=0,
            k=1,
            k33=1,
            # k33overk = True
        )
    npf.set_all_data_external()

    ss = np.ones((nlay, ncpl)) * 1.e-4
    sto = flopy.mf6.ModflowGwfsto(gwf, 
                                    ss=ss, 
                                    iconvert=0,
                                    # steady_state={0: False},
                                    transient={0: True})
    sto.set_all_data_external()

    chd = make_chd(gwf, conservative_tracer=tracer, mup3d_m=mup3d_m)
    # chd.export(os.path.join(ws, f"chd.vtk"), fmt="vtk")

    welout = make_wel_out(gwf, conservative_tracer=tracer, mup3d_m=mup3d_m)
    welin = make_wel_in(gwf, conservative_tracer=tracer, mup3d_m=mup3d_m)
    # disv.export(os.path.join(ws, f"disv.vtk"), fmt="vtk")

        # create the output control
    headfile = f"{flow_modelname}.hds"
    head_filerecord = [headfile]
    budgetfile = f"{flow_modelname}.cbb"
    budget_filerecord = [budgetfile]
    saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")]
    printrecord = [("HEAD", "LAST")]

    oc = flopy.mf6.ModflowGwfoc(
        gwf,
        saverecord=saverecord,
        head_filerecord=head_filerecord,
        budget_filerecord=budget_filerecord,
        printrecord=printrecord,)
    return gwf, sim

In [ ]:
def make_gwt(sim, tracer = 'Cl', mup3d_m=None):
    ne = 0.35 # effective porosity (-) constant across all layers
    long_disp = 0.1 # Longitudinal dispersivity (m)constant across all layers
    disp_tr_vert = long_disp*0.01 # Transverse vertical dispersivity (m) constant across all layers
    disp_tr_hor = long_disp*0.1 # Transverse horizontal dispersivity (m) constant across all layers
    diffc = 0 # diffusion coefficient constant across all layers
    pbulk = 1850 # bulk density in constant across all layers (m/L^3) 
    gwf = sim.get_model('gwf')

    if mup3d_m is not None and tracer is None:
        components = mup3d_m.components
    else:
        components = [tracer]
    # components = ['Cl']
    for comp in components:
        print(f"Setting transport for {comp}")
        model_name = comp
        gwt = flopy.mf6.MFModel(
            sim,
            model_type="gwt6",
            modelname=model_name,
            model_nam_file=f"{model_name}.nam"
        )
        outer_dvclose = 1e-5
        inner_dvclose = 1e-5
        ims = flopy.mf6.ModflowIms(sim, 
                                #    pname="ims", 
                                complexity="complex",
                                outer_dvclose=outer_dvclose,
                                inner_dvclose=inner_dvclose,
                                filename=f"{model_name}.ims")
        sim.register_ims_package(ims, 
                                [model_name])

        dis = gwf.dis
        nlay = dis.nlay.get_data()
        ncpl = dis.ncpl.get_data()
        disv = flopy.mf6.ModflowGwfdisv(gwt,
                                        nlay=nlay,
                                        ncpl=dis.ncpl.get_data(),
                                        nvert=dis.nvert.get_data(),
                                        vertices=dis.vertices.get_data(),
                                        cell2d=dis.cell2d.get_data(),
                                        top=dis.top.get_data(),
                                        botm=dis.botm.get_data(),
                                        filename=f"{model_name}.disv")

        # botms = get_botms(gwf, ws)
        # disv.botm.set_data(botms)
        disv.set_all_data_external()


        if tracer is not None:
            strt = 2.540000e-04
        else:
            strt = mup3d_m.sconc[comp]
        # strt = 2.540000e-04
        ic = flopy.mf6.ModflowGwtic(gwt, strt=strt, 
                                    filename=f"{model_name}.ic")
        ic.set_all_data_external()

        adv = flopy.mf6.ModflowGwtadv(
            gwt,
            scheme="tvd",
        )
        adv.set_all_data_external()

        alpha_l = np.ones(shape=(nlay,ncpl))*long_disp  # Longitudinal dispersivity ($m$)
        alpha_th = np.ones(shape=(nlay,ncpl))*disp_tr_hor  # Transverse horizontal dispersivity ($m$)
        alpha_tv = np.ones(shape=(nlay,ncpl))*disp_tr_vert  # Transverse vertical dispersivity ($m$)

        dsp = flopy.mf6.ModflowGwtdsp(
            gwt,
            xt3d_off=True,
            alh=alpha_l,
            ath1=alpha_th,
            atv = alpha_tv,
            diffc = diffc,
            filename=f"{model_name}.dsp",
        )
        dsp.set_all_data_external()

        sourcerecarray = [
                        ["welin", "aux", model_name],
                        ["welout", "aux", model_name],
                        ["chd", "aux", model_name]
                        ]

        ssm = flopy.mf6.ModflowGwtssm(
                gwt,
                sources=sourcerecarray,
                save_flows=True,
                print_flows=True,
                filename=f"{model_name}.ssm",
            )
        ssm.set_all_data_external()

        if comp=='Tmp':
            # kd = 2*ne/1850
            distcoef = np.ones(shape=(nlay,ncpl))*2.1141E-04
            sorption = "Linear"
            pbulk = 1850
            bulk_density = np.ones(shape=(nlay,ncpl))*pbulk
            ne=0.35
        else:
            distcoef = None
            sorption = None
            bulk_density = None
            ne=0.35
        porosity  = np.ones(shape=(nlay,ncpl))*ne
        
        mst = flopy.mf6.ModflowGwtmst(
            gwt,
            porosity=porosity,
            first_order_decay=None,
            decay = None,
            decay_sorbed=None,
            sorption= sorption,
            bulk_density=bulk_density, 
            distcoef=distcoef, #Kd m3/mg
            sp2 = None,
            filename=f"{model_name}.mst",
        )
        mst.set_all_data_external()
        oc = flopy.mf6.ModflowGwtoc(
            gwt,
            budget_filerecord=f"{model_name}.cbb",
            concentration_filerecord=f"{model_name}.ucn",
            concentrationprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 10, "GENERAL")
                                        ],
            saverecord=[("CONCENTRATION", "ALL"), 
                        ],
            printrecord=[("CONCENTRATION", "LAST"), 
                            ],
        )
        flopy.mf6.ModflowGwfgwt(
            sim,
            exgtype="GWF6-GWT6",
            exgmnamea='gwf',
            exgmnameb=f'{model_name}',
            filename=f"{model_name}.gwfgwt",
        )
        make_obs_pack(gwt)

    sim.write_simulation() 
    return sim

In [ ]:
ws = Path("model")

if ws.exists():
    shutil.rmtree(ws)

gwf, sim = make_gwf(tracer = 'Cl', mup3d_m = None)
sim = make_gwt(sim, tracer = 'Cl', mup3d_m=None)

# pyemu.os_utils.run(f"mf6", cwd=ws)

In [ ]:
def initialize_chemistry(ws, nlay, ncpl):

    perioddata= [(2, 2, 1), (4, 4, 1), (4, 4, 1), (4, 4, 1), (7, 7, 1),
                (7, 7, 1), (7, 7, 1), (7, 7, 1), (14, 14, 1), (14, 14, 1), 
                (15, 15, 1), (13, 13, 1), (14, 14, 1), (14, 14, 1), (14, 14, 1), 
                (21, 21, 1), (35, 35, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
                (28, 28, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
                (35, 35, 1), (35, 35, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
                (35, 35, 1), (35, 35, 1), (28, 28, 1), (28, 28, 1), (28, 28, 1), 
                (28, 28, 1), (35, 35, 1), (35, 35, 1),(28, 28, 1)]
    
    # get chemistry
    solutionsdf = pd.read_csv(os.path.join("data","ic_aq_chem.csv"), index_col = 0)

    ## let's process the injection chem
    injdf = pd.read_csv(os.path.join("data","wellin.csv"), index_col = 0)
    injdf = injdf[['layer'] + solutionsdf.index.tolist()].copy()

    frames = []                     

    for per in injdf.index.unique():
        df = (
            injdf.loc[per].reset_index()      
                .drop(columns='kper')        
                .groupby('layer').mean()    
                .T                          
        )
        # give columns names like  0_1, 0_2, …   (or f"{per}_{layer}")
        df.columns = [f"{per}_{layer}" for layer in df.columns]

        frames.append(df)            

    injdf = pd.concat(frames, axis=1) 
    solutionsdf = pd.concat([solutionsdf, injdf], axis=1)

    solutions = utils.solution_df_to_dict(solutionsdf)

    sol_ic = np.ones((nlay, ncpl), dtype=float)

    solution = mup3d.Solutions(solutions)
    solution.set_ic(sol_ic)
    solution

    excdf = pd.read_csv(os.path.join(os.path.join("data","ic_exchanger.csv")), comment = '#')
    ex_names = {
                "Ca_ex": "CaX2",
                "Fe_ex":"FeX2" ,
                "K_ex":"KX" ,
                "Mg_ex":"MgX2", 
                "Na_ex":"NaX"
                }
    # we need to rename to match the database
    excdf['name'] = excdf['var'].map(ex_names)
    excdf['layer'] -= 1  # convert to zero‑indexed
    excdf = excdf.pivot(index="name", columns="layer", values="value")
    exchanger_dict = excdf.to_dict()
    for k, subdict in exchanger_dict.items():
        for key in subdict:
            subdict[key] = {'m0': subdict[key]}

    exchanger = mup3d.ExchangePhases(exchanger_dict)

    layer_vals = np.arange(1, nlay + 1, dtype=float)   # shape (nlay,)

    # Broadcast to full 3‑D grid
    exchanger_ic = np.ones((1, ncpl))
    exchanger.set_ic(sol_ic)

    # we need to equilibrate the exchangers with the background solution
    # init background solution is the same (number 1) for the whole domain
    # the next array has a length of nlay, one for each layer
    eq_solutions = [1] * nlay
    exchanger.set_equilibrate_solutions(eq_solutions)

    mindf = pd.read_csv(os.path.join(os.path.join("data","ic_surfaces.csv")), comment = '#')
    mindf['value'] = [utils.concentration_volbulk_to_volwater(i,0.35) for i in mindf['value'].values]
    mindf = mindf.pivot(index="var", columns="layer", values="value")

    # only ferrihydrite and orgmatter are in eq
    eq_m0 = utils.solution_df_to_dict(mindf.loc[['Ferrihydrite', "Orgmatter"],:])

    # we need the initial Sat indeces SI
    # following original model init SI is 0
    si = 0

    eq_dic = {}
    #lets add pyrite first
    for ly in range(nlay):
        for key in eq_m0.keys():
            # create a dictionary for each layer with key as the mineral name
            # and values as a dictionary with si and m0
        # si followed by m0 (init moles)
            eq_dic[ly] = {key: {}}
            eq_dic[ly][key]['si'] = si
            eq_dic[ly][key]['m0'] = eq_m0[key][ly]
            # eq_dic[ly+1] = {key: [si, eq_m0[key][ly]] for key in eq_m0.keys()}
    equilibriums = mup3d.EquilibriumPhases(eq_dic)
    equilibriums.set_ic(sol_ic)

    # pyrite is kinetic
    py_m0 = utils.solution_df_to_dict(mindf.loc[["Pyrite"],:])

    # we need the kinetic params for Py
    kin_py_params = [1.600000e+01, 
                        6.700000e-01, 
                        5.000000e-01, 
                        -1.100000e-01]
    kin_dic = {}

    # lets add kinetics for Organic carbon (Orgc)
    kin_orgc_params = [1.570000e-09, 1.670000e-11, 1.000000e-13]
    # orgc also have a custom formula
    orgc_form =  "Orgc -1.0 CH2O 1.0"
    orgc_steps = "8.640000e+04 in 1 steps"

    #lets add pyrite first
    for ly in range(nlay):
        # print(ly+1)
        for key in py_m0.keys():
            kin_dic[ly] = {key: {}}
            kin_dic[ly][key]['m0'] = py_m0[key][ly]
            kin_dic[ly][key]['parms'] = kin_py_params
            # kin_dic[ly+1] = {key: [py_m0[key][ly], kin_py_params]}

    # lets add orgc now with a m0 of zero for all layers
    for key in kin_dic.keys():
        kin_dic[key]['Orgc'] = {}
        kin_dic[key]['Orgc']['m0'] = 1.0
        kin_dic[key]['Orgc']['parms'] = kin_orgc_params
        kin_dic[key]['Orgc']['formula'] = orgc_form
        kin_dic[key]['Orgc']['steps'] = orgc_steps
        # [1.0, kin_orgc_params, orgc_form, orgc_steps]
    kinetics = mup3d.KineticPhases(kin_dic)
    kinetics.set_ic(sol_ic)
    model = mup3d.Mup3d('dizon36',solution, nlay=nlay, ncpl=ncpl)

    # #set model workspace
    model.set_wd(ws)

    # set database
    # shutil copy datab
    shutil.copy(os.path.join("data", f'datab.dat'), os.path.join(model.wd, f'datab.dat'))
    database = os.path.join(f'datab.dat')
    model.set_database(database)

    postfix = os.path.join("data", f'postfix.phqr')
    model.set_postfix(postfix)
    model.set_exchange_phases(exchanger)
    model.set_phases(kinetics)
    model.set_phases(equilibriums)
    # model.set_charge_offset(1e-3)
    tsteps = create_output_pairs(perioddata, output_interval=2)


    model.set_config(
                    reactive_timing='user',
                    reactive_externalio=False,
                    emulator_training_data=False,
                    reactive_tsteps=tsteps
                    )
    model.set_componenth2o(True)
    model.initialize(add_charge_flag=True)
    return model

nlay = gwf.dis.nlay.get_data()
ncpl = gwf.dis.ncpl.get_data()



In [ ]:
ws = Path("model")

if ws.exists():
    shutil.rmtree(ws)
mup3d_m=initialize_chemistry(ws, nlay, ncpl)

In [ ]:
gwf, sim = make_gwf(tracer = None, mup3d_m = mup3d_m)
sim = make_gwt(sim, tracer = None, mup3d_m=mup3d_m)

In [ ]:
# pyemu.os_utils.run(f"mf6rtm", cwd=ws)